문장의 긍, 부정을 판단하는 감성 분류 모델 만들기

In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [2]:
# 테스트 문장과 레이블(긍정:1, 부정:0) 생성

sentences = ['nice great best amazing', 'stop lies', 'pitiful nerd', 'excellent work', 'supreme quality', 'bad', 'highly respectable']
y_train = [1, 0, 0, 1, 1, 0, 1]

In [3]:
# 전처리
# 토큰화

t = Tokenizer()
t.fit_on_texts(sentences)
vocab_size = len(t.word_index) + 1

print(vocab_size)

16


In [4]:
# 정수 인코딩

X_encoded = t.texts_to_sequences(sentences)
print(X_encoded)

[[1, 2, 3, 4], [5, 6], [7, 8], [9, 10], [11, 12], [13], [14, 15]]


In [5]:
# 패딩을 위해 문장 중 가장 길이가 긴 문장의 길이 확인

max_len = max(len(l) for l in X_encoded)
print(max_len)

4


In [6]:
# 패딩

X_train = pad_sequences(X_encoded, maxlen=max_len, padding='post')
y_train = np.array(y_train)
print(X_train)

[[ 1  2  3  4]
 [ 5  6  0  0]
 [ 7  8  0  0]
 [ 9 10  0  0]
 [11 12  0  0]
 [13  0  0  0]
 [14 15  0  0]]


In [7]:
print(y_train)

[1 0 0 1 1 0 1]


In [8]:
# 모델 설계

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

model = Sequential()
model.add(Embedding(vocab_size, 4, input_length=max_len))  # 모든 임베딩 벡터는 4차원
model.add(Flatten())   # Dense의 입력으로 넣기 위함
model.add(Dense(1, activation='sigmoid'))

In [10]:
# 모델 훈련

In [11]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=100, verbose=2)

Epoch 1/100
1/1 - 2s - loss: 0.6896 - acc: 0.5714
Epoch 2/100
1/1 - 0s - loss: 0.6883 - acc: 0.5714
Epoch 3/100
1/1 - 0s - loss: 0.6870 - acc: 0.5714
Epoch 4/100
1/1 - 0s - loss: 0.6856 - acc: 0.5714
Epoch 5/100
1/1 - 0s - loss: 0.6843 - acc: 0.5714
Epoch 6/100
1/1 - 0s - loss: 0.6830 - acc: 0.5714
Epoch 7/100
1/1 - 0s - loss: 0.6816 - acc: 0.7143
Epoch 8/100
1/1 - 0s - loss: 0.6803 - acc: 0.7143
Epoch 9/100
1/1 - 0s - loss: 0.6790 - acc: 0.7143
Epoch 10/100
1/1 - 0s - loss: 0.6776 - acc: 0.7143
Epoch 11/100
1/1 - 0s - loss: 0.6763 - acc: 0.8571
Epoch 12/100
1/1 - 0s - loss: 0.6750 - acc: 0.8571
Epoch 13/100
1/1 - 0s - loss: 0.6736 - acc: 0.8571
Epoch 14/100
1/1 - 0s - loss: 0.6723 - acc: 0.8571
Epoch 15/100
1/1 - 0s - loss: 0.6709 - acc: 0.8571
Epoch 16/100
1/1 - 0s - loss: 0.6696 - acc: 0.8571
Epoch 17/100
1/1 - 0s - loss: 0.6682 - acc: 0.8571
Epoch 18/100
1/1 - 0s - loss: 0.6669 - acc: 1.0000
Epoch 19/100
1/1 - 0s - loss: 0.6655 - acc: 1.0000
Epoch 20/100
1/1 - 0s - loss: 0.6641 - a

## Pretrained Word Embedding Model

  - Word2Vec

  [Pre-trained word vectors of 30+ languages](https://github.com/Kyubyong/wordvectors)

  - FastText

  [Word vectors for 157 languages](https://fasttext.cc/docs/en/crawl-vectors.html)

1) 사전 훈련된 GloVe 사용하기

In [12]:
# 데이터 준비하기

!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

# 하나의 줄당 101개의 값을 가지는 리스트 갖고 있음

--2021-09-08 10:57:14--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-09-08 10:57:14--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-09-08 10:57:14--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [13]:
# 두 개의 줄만 읽어 보기

n = 0
f = open('glove.6B.100d.txt', encoding='utf8')

for line in f:
    word_vector = line.split()  # 각 줄을 읽어와서 word_vector에 저장
    print(word_vector)  # 각 줄을 출력
    word = word_vector[0]  # word_vector에서 첫번째 값만 저장
    print(word)  # word_vector의 첫번째 값만 출력
    n = n+1
    if n == 2:
        break
f.close()

# 101개의 값 중 첫번째 값 'the', ',': 임베딩 벡터가 의미하는 단어, 이후 임베딩 벡처 100개의 차원의 각각의 값

['the', '-0.038194', '-0.24487', '0.72812', '-0.39961', '0.083172', '0.043953', '-0.39141', '0.3344', '-0.57545', '0.087459', '0.28787', '-0.06731', '0.30906', '-0.26384', '-0.13231', '-0.20757', '0.33395', '-0.33848', '-0.31743', '-0.48336', '0.1464', '-0.37304', '0.34577', '0.052041', '0.44946', '-0.46971', '0.02628', '-0.54155', '-0.15518', '-0.14107', '-0.039722', '0.28277', '0.14393', '0.23464', '-0.31021', '0.086173', '0.20397', '0.52624', '0.17164', '-0.082378', '-0.71787', '-0.41531', '0.20335', '-0.12763', '0.41367', '0.55187', '0.57908', '-0.33477', '-0.36559', '-0.54857', '-0.062892', '0.26584', '0.30205', '0.99775', '-0.80481', '-3.0243', '0.01254', '-0.36942', '2.2167', '0.72201', '-0.24978', '0.92136', '0.034514', '0.46745', '1.1079', '-0.19358', '-0.074575', '0.23353', '-0.052062', '-0.22044', '0.057162', '-0.15806', '-0.30798', '-0.41625', '0.37972', '0.15006', '-0.53212', '-0.2055', '-1.2526', '0.071624', '0.70565', '0.49744', '-0.42063', '0.26148', '-1.538', '-0.30223

In [14]:
print(type(word_vector))
print(len(word_vector))

<class 'list'>
101


In [15]:
# 모든 임베딩 벡터들을 dictionary type(key:value)으로 불러오기

In [16]:
embedding_dict = dict()

f = open('glove.6B.100d.txt', encoding="utf8")

for line in f:
    word_vector = line.split()
    word = word_vector[0]
    word_vector_arr = np.asarray(word_vector[1:], dtype='float32') # 100개의 값을 가지는 array로 변환
    embedding_dict[word] = word_vector_arr
f.close()
print('%s개의 Embedding vector가 있습니다.' % len(embedding_dict))

400000개의 Embedding vector가 있습니다.


In [17]:
# 임의의 단어 'respectable'에 대해서 임베딩 벡터를 출력 해보기

In [18]:
print(embedding_dict['respectable'])
print(len(embedding_dict['respectable']))

# 각각의 벡터 값들과 총 개수 출력

[-0.049773   0.19903    0.10585    0.1391    -0.32395    0.44053
  0.3947    -0.22805   -0.25793    0.49768    0.15384   -0.08831
  0.0782    -0.8299    -0.037788   0.16772   -0.45197   -0.17085
  0.74756    0.98256    0.81872    0.28507    0.16178   -0.48626
 -0.006265  -0.92469   -0.30625   -0.067318  -0.046762  -0.76291
 -0.0025264 -0.018795   0.12882   -0.52457    0.3586     0.43119
 -0.89477   -0.057421  -0.53724    0.25587    0.55195    0.44698
 -0.24252    0.29946    0.25776   -0.8717     0.68426   -0.05688
 -0.1848    -0.59352   -0.11227   -0.57692   -0.013593   0.18488
 -0.32507   -0.90171    0.17672    0.075601   0.54896   -0.21488
 -0.54018   -0.45882   -0.79536    0.26331    0.18879   -0.16363
  0.3975     0.1099     0.1164    -0.083499   0.50159    0.35802
  0.25677    0.088546   0.42108    0.28674   -0.71285   -0.82915
  0.15297   -0.82712    0.022112   1.067     -0.31776    0.1211
 -0.069755  -0.61327    0.27308   -0.42638   -0.085084  -0.17694
 -0.0090944  0.1109     0.

In [19]:
# 훈련 데이터의 단어 집합의 모든 단어에 대해서 사전 훈련된 GloVe의 임베딩 벡터들을 맵핑

In [20]:
embedding_matrix = np.zeros((vocab_size, 100))
# 단어 집합 크기의 행과 100개의 열을 가지는 행렬 생성. 값은 전부 0으로 채워진다.

np.shape(embedding_matrix)

(16, 100)

In [21]:
print(t.word_index.items())

dict_items([('nice', 1), ('great', 2), ('best', 3), ('amazing', 4), ('stop', 5), ('lies', 6), ('pitiful', 7), ('nerd', 8), ('excellent', 9), ('work', 10), ('supreme', 11), ('quality', 12), ('bad', 13), ('highly', 14), ('respectable', 15)])


In [22]:
for word, i in t.word_index.items(): # 훈련 데이터의 단어 집합에서 단어를 1개씩 꺼내온다.
    temp = embedding_dict.get(word) # 단어(key) 해당되는 임베딩 벡터의 100개의 값(value)를 임시 변수에 저장
    if temp is not None:
        embedding_matrix[i] = temp # 임수 변수의 값을 단어와 맵핑되는 인덱스의 행에 삽입

In [23]:
# 임베딩 층 생성 : 모델 설계

In [24]:
model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_len, trainable=False)

model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [25]:
# 모델 훈련

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=100, verbose=2)

Epoch 1/100
1/1 - 1s - loss: 0.7013 - acc: 0.4286
Epoch 2/100
1/1 - 0s - loss: 0.6815 - acc: 0.5714
Epoch 3/100
1/1 - 0s - loss: 0.6624 - acc: 0.5714
Epoch 4/100
1/1 - 0s - loss: 0.6440 - acc: 0.7143
Epoch 5/100
1/1 - 0s - loss: 0.6262 - acc: 0.7143
Epoch 6/100
1/1 - 0s - loss: 0.6091 - acc: 0.7143
Epoch 7/100
1/1 - 0s - loss: 0.5926 - acc: 0.7143
Epoch 8/100
1/1 - 0s - loss: 0.5768 - acc: 0.7143
Epoch 9/100
1/1 - 0s - loss: 0.5614 - acc: 0.7143
Epoch 10/100
1/1 - 0s - loss: 0.5467 - acc: 0.7143
Epoch 11/100
1/1 - 0s - loss: 0.5324 - acc: 0.7143
Epoch 12/100
1/1 - 0s - loss: 0.5187 - acc: 0.7143
Epoch 13/100
1/1 - 0s - loss: 0.5054 - acc: 0.7143
Epoch 14/100
1/1 - 0s - loss: 0.4926 - acc: 0.7143
Epoch 15/100
1/1 - 0s - loss: 0.4802 - acc: 0.7143
Epoch 16/100
1/1 - 0s - loss: 0.4682 - acc: 0.8571
Epoch 17/100
1/1 - 0s - loss: 0.4566 - acc: 0.8571
Epoch 18/100
1/1 - 0s - loss: 0.4454 - acc: 0.8571
Epoch 19/100
1/1 - 0s - loss: 0.4345 - acc: 0.8571
Epoch 20/100
1/1 - 0s - loss: 0.4240 - a

2) 사전 훈련된 Word2Vec 사용하기

In [26]:
import gensim

In [27]:
# 데이터 준비하기

In [28]:
# 현재 위치에 구글의 사전 훈련된 Word2Vec을 다운로드

!wget "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2021-09-08 11:00:39--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.99.78
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.99.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  46.3MB/s    in 45s     

2021-09-08 11:01:24 (34.9 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [29]:
# 구글의 사전 훈련된 Word2vec 모델 로드

word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

In [30]:
# 모델 크기 확인

print(word2vec_model.vectors.shape)

(3000000, 300)


In [31]:
# 임베딩 행렬 생성 : 모두 0으로(이번 문제의 단어는 총 16개이므로, 16 × 300의 크기를 가진 행렬)

embedding_matrix = np.zeros((vocab_size, 300))

# 단어 집합 크기의 행과 300개의 열을 가지는 행렬 생성. 값은 전부 0으로 채워진다.
np.shape(embedding_matrix)

(16, 300)

In [32]:
# 특정 단어 입력 받기

def get_vector(word):
    if word in word2vec_model:
        return word2vec_model[word]
    else:
        return None

# 해당 단어 임베딩 벡터 리턴, 없으면 None 리턴

In [33]:
# 단어 집합으로부터 단어를 1개씩 호출하여 word2vec_model에 해당 단어의 임베딩 벡터값이 존재하는지 확인
# 존재한다면 임베딩 행렬에 해당 단어의 인덱스 위치의 행에 임베딩 벡터의 값을 저장

for word, i in t.word_index.items(): # 훈련 데이터의 단어 집합에서 단어와 정수 인덱스를 1개씩 꺼내온다.
    temp = get_vector(word) # 단어(key) 해당되는 임베딩 벡터의 300개의 값(value)를 임시 변수에 저장
    if temp is not None: # 만약 None이 아니라면 임베딩 벡터의 값을 리턴받은 것이므로
        embedding_matrix[i] = temp # 해당 단어 위치의 행에 벡터의 값을 저장한다.

In [34]:
#  'nice'의 임베딩 벡터값 확인을 통해 맴핑 결과 확인

print(word2vec_model['nice'])

# 'nice'는 현재 단어 집합에서 몇 번 인덱스를 가지는지 확인
print('단어 nice의 정수 인덱스 :', t.word_index['nice'])

# 해당 인덱스의 임베딩 벡터 값 확인
print(embedding_matrix[1])

[ 0.15820312  0.10595703 -0.18945312  0.38671875  0.08349609 -0.26757812
  0.08349609  0.11328125 -0.10400391  0.17871094 -0.12353516 -0.22265625
 -0.01806641 -0.25390625  0.13183594  0.0859375   0.16113281  0.11083984
 -0.11083984 -0.0859375   0.0267334   0.34570312  0.15136719 -0.00415039
  0.10498047  0.04907227 -0.06982422  0.08642578  0.03198242 -0.02844238
 -0.15722656  0.11865234  0.36132812  0.00173187  0.05297852 -0.234375
  0.11767578  0.08642578 -0.01123047  0.25976562  0.28515625 -0.11669922
  0.38476562  0.07275391  0.01147461  0.03466797  0.18164062 -0.03955078
  0.04199219  0.01013184 -0.06054688  0.09765625  0.06689453  0.14648438
 -0.12011719  0.08447266 -0.06152344  0.06347656  0.3046875  -0.35546875
 -0.2890625   0.19628906 -0.33203125 -0.07128906  0.12792969  0.09619141
 -0.12158203 -0.08691406 -0.12890625  0.27734375  0.265625    0.1796875
  0.12695312  0.06298828 -0.34375    -0.05908203  0.0456543   0.171875
  0.08935547  0.14648438 -0.04638672 -0.00842285 -0.0279

In [35]:
# 모델 설계

model = Sequential()
e = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_len, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

# 모델 훈련

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=100, verbose=2)

Epoch 1/100
1/1 - 1s - loss: 0.6676 - acc: 0.5714
Epoch 2/100
1/1 - 0s - loss: 0.6495 - acc: 0.5714
Epoch 3/100
1/1 - 0s - loss: 0.6320 - acc: 0.5714
Epoch 4/100
1/1 - 0s - loss: 0.6150 - acc: 0.7143
Epoch 5/100
1/1 - 0s - loss: 0.5986 - acc: 0.8571
Epoch 6/100
1/1 - 0s - loss: 0.5828 - acc: 1.0000
Epoch 7/100
1/1 - 0s - loss: 0.5674 - acc: 1.0000
Epoch 8/100
1/1 - 0s - loss: 0.5526 - acc: 1.0000
Epoch 9/100
1/1 - 0s - loss: 0.5383 - acc: 1.0000
Epoch 10/100
1/1 - 0s - loss: 0.5245 - acc: 1.0000
Epoch 11/100
1/1 - 0s - loss: 0.5112 - acc: 1.0000
Epoch 12/100
1/1 - 0s - loss: 0.4983 - acc: 1.0000
Epoch 13/100
1/1 - 0s - loss: 0.4859 - acc: 1.0000
Epoch 14/100
1/1 - 0s - loss: 0.4739 - acc: 1.0000
Epoch 15/100
1/1 - 0s - loss: 0.4624 - acc: 1.0000
Epoch 16/100
1/1 - 0s - loss: 0.4512 - acc: 1.0000
Epoch 17/100
1/1 - 0s - loss: 0.4404 - acc: 1.0000
Epoch 18/100
1/1 - 0s - loss: 0.4299 - acc: 1.0000
Epoch 19/100
1/1 - 0s - loss: 0.4198 - acc: 1.0000
Epoch 20/100
1/1 - 0s - loss: 0.4100 - a